In [12]:
url_main = 'https://www.beeradvocate.com'
db_path = '/home/joseph/Documents/SQL-Lessons/BeerAdvocateDB'

In [13]:
def get_page(url):
    import requests
    from bs4 import BeautifulSoup as bs

    result = requests.get(url)
    
    try:
        page = bs(result.content, 'lxml')
        return page
    except:
        print('Request failed')

In [17]:
def get_top_beer_table():
    top_beer_list = []
    
    # sqlite3 connection
    import sqlite3
    cnx = sqlite3.connect(db_path)
    cursor = cnx.cursor()

    # Create table: ba_top_250_beers
    cursor.execute('''CREATE TABLE ba_top_250_beers (Name, Style, Brewery, Score, Ratings, City, State, Country);''')
    
    # soup
    url_list = url_main + '/lists/top/'
    page = get_page(url_list)
    table_rows = page.find_all('tr') # get top beers table
    top_beers = table_rows[2:] # interested only in rows with top beers info
    #top_beers[0]
    
    for a in range(249):
        anchors = top_beers[a].find_all('a') # get all anchors within each top beer row
        # Ratings
        rating = top_beers[a].find_all('td')

         # Link to biz info
        url_info = url_main + anchors[1].get('href') #'Link to brewery info => pass to function that gets addresses'
        location_list = get_biz_info(url_info)
        
        # Beer name, Brewery, Style etc.
        """entry = {
            'Name': anchors[0].get_text(), 
            'Brewery': anchors[1].get_text(), 
            'Style': anchors[2].get_text(), 
            'Score': float(rating[2].get_text()), 
            'Ratings': int(rating[3].get_text().replace(',', '')), 
            'City': location_list[0],
            'State': location_list[1],
            'Country': location_list[2],
            #'Website': location_list[3]
        }"""
    
        #top_beer_list.append(entry)
        cursor.execute("INSERT INTO ba_top_250_beers VALUES (anchors[0].get_text(), anchors[1].get_text(), anchors[1].get_text(), float(rating[2].get_text()), int(rating[3].get_text().replace(',', '')), location_list[0], location_list[2], location_list[3]);")
        
    # Close connection, commit table   
    cnx.commit()
    cnx.close()
    return top_beer_list

In [15]:
def get_biz_info(url):
    page = get_page(url)
    info_box = page.find('div', {'id': 'info_box'})
    info_anchors = info_box.find_all('a')
    
    location_list = []
    
    city = info_anchors[0].get_text() # city
    state = info_anchors[1].get_text() # state
    country = info_anchors[2].get_text() # country
    #website = info_anchors[4].get('href') # website
    
    location_list.extend((city, state, country))
    return location_list

In [18]:
top_beer_list = get_top_beer_table()

In [11]:
top_beer_list[0:3]

[{'Name': 'Kentucky Brunch Brand Stout',
  'Brewery': 'Toppling Goliath Brewing Company',
  'Style': 'American Double / Imperial Stout',
  'Score': 4.84,
  'Ratings': 696,
  'City': 'Decorah',
  'State': 'Iowa',
  'Country': 'United States'},
 {'Name': 'Marshmallow Handjee',
  'Brewery': '3 Floyds Brewing Co.',
  'Style': 'Russian Imperial Stout',
  'Score': 4.74,
  'Ratings': 1601,
  'City': 'Munster',
  'State': 'Indiana',
  'Country': 'United States'},
 {'Name': 'Barrel-Aged Abraxas',
  'Brewery': 'Perennial Artisan Ales',
  'Style': 'American Double / Imperial Stout',
  'Score': 4.74,
  'Ratings': 1430,
  'City': 'Saint Louis',
  'State': 'Missouri',
  'Country': 'United States'}]

In [12]:
import pandas as pd
pd_tb_list = pd.DataFrame(top_beer_list, columns=['Name', 'Style', 'Brewery', \
                                                  'Score', 'Ratings', 'City', \
                                                  'State', 'Country'])

In [14]:
pd_tb_list.tail(10)

,Name,Style,Brewery,Score,Ratings,City,State,Country
239,Barrel Aged Double Shot Double Black,American Black Ale,Bent Paddle Brewing Co.,4.49,277,Duluth,Minnesota,United States
240,Trickery,Milk / Sweet Stout,Hardywood Park Craft Brewery,4.45,483,Richmond,Virginia,United States
241,Mélange No. 3,American Strong Ale,The Bruery,4.41,1985,Placentia,California,United States
242,Yellow Rose,American IPA,The Lone Pint Brewery,4.41,1838,Magnolia,Texas,United States
243,Rue D'Floyd,American Porter,The Bruery,4.43,743,Placentia,California,United States
244,Dreadnaught IPA,American Double / Imperial IPA,3 Floyds Brewing Co.,4.40,6095,Munster,Indiana,United States
245,Citra Showers,American Double / Imperial IPA,Noble Ale Works,4.44,554,Anaheim,California,United States
246,Leon,English Barleywine,Cigar City Brewing,4.44,553,Tampa,Florida,United States
247,Master Of Karate,American Double / Imperial IPA,Aslin Beer Company,4.46,381,Herndon,Virginia,United States
248,Oro Negro,American Double / Imperial Stout,Jackie O's Pub & Brewery,4.42,913,Athens,Ohio,United States
